In [1]:
import ctypes
import Cython
import numpy as np
%load_ext Cython

In [2]:
%%cython
cdef extern from "/home/bzr0014/git/exploit-development/drafts/c_test/ctypes/sum.h":
    int our_function(int numbers, int* numbers)

cpdef our_function_cython(int[:] numbers):
    cdef int n = numbers.shape[0]
    return our_function(n,  &numbers[0])

In [15]:
%%cython
from cython.parallel cimport prange
from cython cimport wraparound, boundscheck
cpdef our_function_cython2(int[:] numbers):
    cdef int i;
    cdef int num_numbers = numbers.shape[0];
    cdef int sum = 0;
    with wraparound(False), boundscheck(False):
        for i in prange(num_numbers, nogil=True):
            sum += numbers[i];
    return sum;

/home/bzr0014/envs/hack/local/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/bzr0014/.cache/ipython/cython/_cython_magic_4488488e5a1f0bf5aaa0221a15f002e9.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [3]:
_sum = ctypes.CDLL('./libsum.so')
_sum.our_function.argtypes = (ctypes.c_int, ctypes.POINTER(ctypes.c_int))

def our_function_ctype(numbers):
    global _sum
    num_numbers = len(numbers)
    array_type = ctypes.c_int * num_numbers
    result = _sum.our_function(ctypes.c_int(num_numbers), array_type(*numbers))
    return int(result)

In [17]:
a = np.random.randint(0, 1000, 10000).astype('int32')

In [18]:
%timeit our_function_ctype(a)

100 loops, best of 3: 2.15 ms per loop


In [19]:
%timeit np.sum(a)

The slowest run took 9.03 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 9.85 µs per loop


In [20]:
%timeit our_function_cython(a)

100000 loops, best of 3: 3.91 µs per loop


In [21]:
%timeit our_function_cython2(a)

100000 loops, best of 3: 4.88 µs per loop
